In [2]:
"""
Conductance Quantization Analysis
==================================
Analyzes PicoScope CSV traces from gold wire break-junction experiments
to extract conductance quanta (G0 = 2e²/h).

The script:
  1. Loads all PicoScope CSV files from a directory
  2. Automatically detects "step regions" — transitions where the wire
     passes through quantized conductance levels (not just rail-to-open jumps)
  3. Converts voltage to conductance in units of G0
  4. Produces four figures:
     - Overview of all traces with expected quantum levels marked
     - Zoomed-in step regions plotted as G/G0 vs time
     - Conductance histogram from step regions
     - Voltage-domain view of all step regions

Circuit model:
  Voltage divider:  V_G = V_SUPPLY * R_D2 / (R_D1 + R_D2)
  Inverting amp:    V_out = V_open - (R_A / R_G) * V_G
  Conductance:      G = (V_open - V_out) / (R_A * V_G)

Usage:
  1. Set DATA_DIR to the folder containing your CSV files
  2. Set FILE_PREFIX to match your filenames (e.g. "20260211-0008_")
  3. Adjust circuit parameters (R_D1, R_D2, R_A, V_SUPPLY) to match your setup
  4. Run:  python conductance_quantization_analysis.py
"""

import csv
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')


In [3]:
# ============================================================
# CONFIGURATION — adjust these to match your setup
# ============================================================

# Circuit parameters
R_D1 = 10230.0       # Voltage divider resistor 1 (Ω)
R_D2 = 99.8          # Voltage divider resistor 2 (Ω)
R_A  = 325900.0      # Feedback resistor of inverting amplifier (Ω)
V_SUPPLY = 5.0       # Supply voltage to voltage divider (V)

# Data location
DATA_DIR = Path(".")              # Folder containing CSV files
FILE_PREFIX = "working_traces/trace_"    # Common prefix of CSV filenames
OUTPUT_DIR = Path("./output")     # Where to save figures

# Step detection parameters
MIN_STEP_POINTS = 15    # Minimum data points for a region to count as a "step"
CONTEXT_POINTS = 30     # Extra points to include on each side of a step region
RAIL_THRESHOLD = -8.5   # Voltages below this are considered op-amp railing (V)
OPEN_MARGIN = 0.3       # Margin below V_open to start counting as "intermediate" (V)

In [4]:
# ============================================================
# Derived constants (computed automatically)
# ============================================================

V_G = V_SUPPLY * R_D2 / (R_D1 + R_D2)   # Voltage across gold wires (V)
G0 = 2 * (1.602176634e-19)**2 / 6.62607015e-34  # Conductance quantum (S)
V_STEP = R_A * V_G * G0  # Voltage shift per conductance quantum (V)

In [5]:

# ============================================================
# Functions
# ============================================================

def load_trace(filepath):
    """
    Load a PicoScope 7 CSV file.

    Returns time (ms) and voltage (V) arrays. Points where the scope
    recorded -∞ (op-amp fully railed) are stored as NaN.
    """
    times, volts = [], []
    with open(filepath, 'r', encoding='utf-8-sig') as f:
        reader = csv.reader(f)
        next(reader)  # "Time,Channel A"
        next(reader)  # "(ms),(V)"
        next(reader)  # blank line
        for row in reader:
            if len(row) >= 2:
                try:
                    t = float(row[0])
                except (ValueError, OverflowError):
                    continue
                try:
                    v = float(row[1])
                except (ValueError, OverflowError):
                    v = np.nan  # -∞ rows become NaN
                times.append(t)
                volts.append(v)
    return np.array(times), np.array(volts)


def voltage_to_conductance(v, v_open):
    """Convert measured output voltage to conductance in units of G0."""
    return (v_open - v) / V_STEP


def find_step_regions(t, v, v_open):
    """
    Find contiguous regions where the voltage is in the "intermediate" range
    (between open-circuit baseline and the op-amp rail). These are the
    transition regions that may contain conductance quantization steps.

    Returns a list of (start_index, end_index) tuples.
    """
    # Replace NaN with a very negative value for masking purposes
    v_safe = np.where(np.isnan(v), -100, v)
    intermediate = (v_safe < v_open - OPEN_MARGIN) & (v_safe > RAIL_THRESHOLD)

    # Find contiguous regions
    regions = []
    in_region = False
    start = 0
    for i in range(len(intermediate)):
        if intermediate[i] and not in_region:
            start = i
            in_region = True
        elif not intermediate[i] and in_region:
            if i - start > 3:  # at least a few points
                regions.append((
                    max(0, start - CONTEXT_POINTS),
                    min(len(t), i + CONTEXT_POINTS)
                ))
            in_region = False
    if in_region and len(t) - start > 3:
        regions.append((max(0, start - CONTEXT_POINTS), len(t)))

    # Merge overlapping regions
    merged = []
    for r in regions:
        if merged and r[0] <= merged[-1][1]:
            merged[-1] = (merged[-1][0], max(merged[-1][1], r[1]))
        else:
            merged.append(r)

    # Keep only regions with enough intermediate data points
    good = []
    for s, e in merged:
        v_r = v[s:e]
        v_valid = v_r[~np.isnan(v_r)]
        n_intermediate = np.sum((v_valid > RAIL_THRESHOLD) & (v_valid < v_open - OPEN_MARGIN))
        if n_intermediate >= MIN_STEP_POINTS:
            good.append((s, e))

    return good


def detect_open_circuit_baseline(all_traces):
    """
    Automatically detect the open-circuit voltage baseline from the data.
    This is the most common high voltage value (wire fully broken).
    """
    high_volts = []
    for t, v in all_traces:
        v_valid = v[~np.isnan(v)]
        high_volts.extend(v_valid[v_valid > 2.0].tolist())
    if high_volts:
        return np.median(high_volts)
    else:
        return 2.94  # fallback default


In [6]:


# ============================================================
# Main analysis
# ============================================================

def main():
    OUTPUT_DIR.mkdir(exist_ok=True)

    # Find all CSV files
    files = sorted(DATA_DIR.glob(f"{FILE_PREFIX}*.csv"))
    if not files:
        print(f"ERROR: No files found matching '{DATA_DIR / (FILE_PREFIX + '*.csv')}'")
        print(f"  Check DATA_DIR and FILE_PREFIX at the top of the script.")
        return

    print(f"Found {len(files)} trace files in {DATA_DIR.resolve()}")
    print(f"\nCircuit parameters:")
    print(f"  R_D1    = {R_D1:.1f} Ω")
    print(f"  R_D2    = {R_D2:.1f} Ω")
    print(f"  R_A     = {R_A:.1f} Ω")
    print(f"  V_supply = {V_SUPPLY:.1f} V")
    print(f"  V_G     = {V_G*1000:.2f} mV")
    print(f"  G0      = {G0:.4e} S  ({1/G0:.0f} Ohm)")
    print(f"  dV/G0   = {V_STEP:.3f} V")

    # Load all traces
    all_traces = []
    for fpath in files:
        t, v = load_trace(fpath)
        label = fpath.stem.replace(FILE_PREFIX, '#')
        all_traces.append((t, v, label, fpath))

    # Detect open-circuit baseline
    V_OPEN = detect_open_circuit_baseline([(t, v) for t, v, _, _ in all_traces])
    print(f"\n  Detected open-circuit baseline: {V_OPEN:.3f} V")

    print(f"\nExpected voltage levels:")
    for n in range(0, 8):
        v_exp = V_OPEN - n * V_STEP
        label = 'open' if n == 0 else f'{n}G0'
        print(f"  n={n} ({label}): V = {v_exp:.3f} V")

    # ---- Find step regions in all traces ----
    all_step_regions = []  # list of (time_array, volt_array, trace_label)
    all_g_values = []      # conductance values from all step regions

    print(f"\nAnalyzing traces for step regions...")
    for t, v, label, fpath in all_traces:
        regions = find_step_regions(t, v, V_OPEN)
        if regions:
            print(f"  {label}: {len(regions)} step region(s)")
        for s, e in regions:
            t_r, v_r = t[s:e], v[s:e]
            all_step_regions.append((t_r, v_r, label))

            # Collect intermediate conductance values for histogram
            v_valid = v_r[~np.isnan(v_r)]
            step_v = v_valid[(v_valid > RAIL_THRESHOLD) & (v_valid < V_OPEN - OPEN_MARGIN)]
            all_g_values.extend(voltage_to_conductance(step_v, V_OPEN).tolist())

    all_g_values = np.array(all_g_values)
    print(f"\nTotal step regions found: {len(all_step_regions)}")
    print(f"Total conductance data points: {len(all_g_values)}")

    # ================================================================
    # FIGURE 1: Overview of all traces
    # ================================================================
    n_traces = len(all_traces)
    ncols = min(5, n_traces)
    nrows = int(np.ceil(n_traces / ncols))

    fig1, axes1 = plt.subplots(nrows, ncols, figsize=(4*ncols, 3*nrows))
    if n_traces == 1:
        axes1 = np.array([[axes1]])
    elif nrows == 1:
        axes1 = axes1.reshape(1, -1)
    elif ncols == 1:
        axes1 = axes1.reshape(-1, 1)

    for idx, (t, v, label, _) in enumerate(all_traces):
        row, col = divmod(idx, ncols)
        ax = axes1[row, col]
        ax.plot(t, v, 'b-', linewidth=0.3)
        for n in range(1, 6):
            v_exp = V_OPEN - n * V_STEP
            if v_exp > -10:
                colors = ['red', 'orange', 'green', 'purple', 'brown']
                ax.axhline(y=v_exp, color=colors[n-1], linestyle='--',
                           alpha=0.4, linewidth=0.7)
        ax.axhline(y=V_OPEN, color='gray', linestyle='--', alpha=0.3, linewidth=0.5)
        ax.set_ylim(-11, 5)
        ax.set_title(label, fontsize=8)
        ax.tick_params(labelsize=6)

    for idx in range(n_traces, nrows * ncols):
        row, col = divmod(idx, ncols)
        axes1[row, col].set_visible(False)

    fig1.suptitle('All Traces (dashed = expected nG0 levels)', fontsize=12)
    fig1.tight_layout()
    fig1.savefig(OUTPUT_DIR / 'fig1_all_traces.png', dpi=150)
    plt.close(fig1)
    print(f"\nSaved: {OUTPUT_DIR / 'fig1_all_traces.png'}")

    # ================================================================
    # FIGURE 2: Step regions as conductance (G/G0) vs time
    # ================================================================
    n_regions = len(all_step_regions)
    if n_regions > 0:
        n_show = min(n_regions, 12)
        ncols2 = min(3, n_show)
        nrows2 = int(np.ceil(n_show / ncols2))

        fig2, axes2 = plt.subplots(nrows2, ncols2, figsize=(5*ncols2, 3*nrows2))
        if n_show == 1:
            axes2 = np.array([[axes2]])
        elif nrows2 == 1:
            axes2 = axes2.reshape(1, -1)
        elif ncols2 == 1:
            axes2 = axes2.reshape(-1, 1)

        for idx in range(n_show):
            t_r, v_r, label = all_step_regions[idx]
            g_r = np.clip(voltage_to_conductance(v_r, V_OPEN), -0.5, 10)

            row, col = divmod(idx, ncols2)
            ax = axes2[row, col]
            ax.plot(t_r, g_r, 'b-', linewidth=1.0)
            for n in range(1, 7):
                ax.axhline(y=n, color='red', linestyle='--', alpha=0.4, linewidth=0.7)
            ax.axhline(y=0, color='gray', linestyle='-', alpha=0.3)
            ax.set_ylim(-0.5, 9)
            ax.set_xlabel('t (ms)', fontsize=9)
            ax.set_ylabel('G/G0', fontsize=9)
            ax.set_title(f'{label}: t=[{t_r[0]:.2f}, {t_r[-1]:.2f}] ms', fontsize=9)
            ax.grid(True, alpha=0.2)

        for idx in range(n_show, nrows2 * ncols2):
            row, col = divmod(idx, ncols2)
            axes2[row, col].set_visible(False)

        fig2.suptitle('Step Regions (Conductance vs Time)', fontsize=13)
        fig2.tight_layout()
        fig2.savefig(OUTPUT_DIR / 'fig2_step_regions.png', dpi=150)
        plt.close(fig2)
        print(f"Saved: {OUTPUT_DIR / 'fig2_step_regions.png'}")

    # ================================================================
    # FIGURE 3: Conductance histogram
    # ================================================================
    if len(all_g_values) > 0:
        fig3, ax3 = plt.subplots(figsize=(10, 5))
        ax3.hist(all_g_values, bins=100, range=(0, 8),
                 color='steelblue', edgecolor='none', alpha=0.85)
        for n in range(1, 8):
            ax3.axvline(x=n, color='red', linestyle='--', alpha=0.6, linewidth=1)
            ax3.text(n, ax3.get_ylim()[1] * 0.9, f'{n}G0',
                     fontsize=10, ha='center', color='red')
        ax3.set_xlabel('Conductance (G0)', fontsize=12)
        ax3.set_ylabel('Counts', fontsize=12)
        ax3.set_title(f'Conductance Histogram -- {len(all_g_values)} points '
                       f'from {len(all_step_regions)} step regions', fontsize=13)
        ax3.grid(True, alpha=0.2)
        fig3.tight_layout()
        fig3.savefig(OUTPUT_DIR / 'fig3_conductance_histogram.png', dpi=150)
        plt.close(fig3)
        print(f"Saved: {OUTPUT_DIR / 'fig3_conductance_histogram.png'}")

        # Peak analysis
        print(f"\nConductance peak analysis:")
        for n in range(1, 7):
            near = all_g_values[(all_g_values > n - 0.25) & (all_g_values < n + 0.25)]
            if len(near) > 0:
                print(f"  {n}G0: {len(near)} pts, mean = {near.mean():.3f} G0")
    else:
        print("\nNo step data found -- check your traces or adjust detection parameters.")

    # ================================================================
    # FIGURE 4: Step regions in voltage domain
    # ================================================================
    if n_regions > 0:
        n_show = min(n_regions, 12)
        ncols4 = min(3, n_show)
        nrows4 = int(np.ceil(n_show / ncols4))

        fig4, axes4 = plt.subplots(nrows4, ncols4, figsize=(5*ncols4, 3*nrows4))
        if n_show == 1:
            axes4 = np.array([[axes4]])
        elif nrows4 == 1:
            axes4 = axes4.reshape(1, -1)
        elif ncols4 == 1:
            axes4 = axes4.reshape(-1, 1)

        for idx in range(n_show):
            t_r, v_r, label = all_step_regions[idx]
            row, col = divmod(idx, ncols4)
            ax = axes4[row, col]
            ax.plot(t_r, v_r, 'b-', linewidth=1.0)

            v_valid = v_r[~np.isnan(v_r)]
            ymin = max(v_valid.min() - 0.5, -11) if len(v_valid) > 0 else -11
            ymax = min(v_valid.max() + 0.5, 5) if len(v_valid) > 0 else 5

            for n in range(0, 8):
                ve = V_OPEN - n * V_STEP
                if ve > ymin and ve < ymax:
                    color = ['gray','red','orange','green','purple','brown','pink','cyan'][n]
                    ax.axhline(y=ve, color=color, linestyle='--', alpha=0.5, linewidth=0.8)

            ax.set_ylim(ymin, ymax)
            ax.set_xlabel('t (ms)', fontsize=9)
            ax.set_ylabel('V_out (V)', fontsize=9)
            ax.set_title(f'{label}: t=[{t_r[0]:.2f}, {t_r[-1]:.2f}] ms', fontsize=9)
            ax.grid(True, alpha=0.2)

        for idx in range(n_show, nrows4 * ncols4):
            row, col = divmod(idx, ncols4)
            axes4[row, col].set_visible(False)

        fig4.suptitle('Step Regions (Voltage vs Time)', fontsize=13)
        fig4.tight_layout()
        fig4.savefig(OUTPUT_DIR / 'fig4_step_regions_voltage.png', dpi=150)
        plt.close(fig4)
        print(f"Saved: {OUTPUT_DIR / 'fig4_step_regions_voltage.png'}")

    # ---- Summary ----
    print(f"\n{'='*60}")
    print(f"SUMMARY")
    print(f"{'='*60}")
    print(f"Traces analyzed:        {len(all_traces)}")
    print(f"Step regions found:     {len(all_step_regions)}")
    print(f"Conductance data pts:   {len(all_g_values)}")
    print(f"Open-circuit baseline:  {V_OPEN:.3f} V")
    print(f"Voltage per G0 step:    {V_STEP:.3f} V")
    print(f"Output directory:       {OUTPUT_DIR.resolve()}")


In [7]:

if __name__ == '__main__':
    main()

Found 39 trace files in /Users/zhangyongqi/Desktop/PHY4400/Lab1/data

Circuit parameters:
  R_D1    = 10230.0 Ω
  R_D2    = 99.8 Ω
  R_A     = 325900.0 Ω
  V_supply = 5.0 V
  V_G     = 48.31 mV
  G0      = 7.7481e-05 S  (12906 Ohm)
  dV/G0   = 1.220 V

  Detected open-circuit baseline: 2.941 V

Expected voltage levels:
  n=0 (open): V = 2.941 V
  n=1 (1G0): V = 1.722 V
  n=2 (2G0): V = 0.502 V
  n=3 (3G0): V = -0.718 V
  n=4 (4G0): V = -1.938 V
  n=5 (5G0): V = -3.158 V
  n=6 (6G0): V = -4.377 V
  n=7 (7G0): V = -5.597 V

Analyzing traces for step regions...
  trace_1: 3 step region(s)
  trace_10: 4 step region(s)
  trace_11: 7 step region(s)
  trace_12: 6 step region(s)
  trace_13: 2 step region(s)
  trace_14: 1 step region(s)
  trace_15: 1 step region(s)
  trace_16: 1 step region(s)
  trace_17: 2 step region(s)
  trace_18: 1 step region(s)
  trace_19: 2 step region(s)
  trace_2: 3 step region(s)
  trace_20: 1 step region(s)
  trace_21: 2 step region(s)
  trace_22: 3 step region(s)
  